In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from tqdm import tqdm
import re
from pprint import pprint

In [2]:
from pymongo import MongoClient
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['vacancydb']
vacancydb = db.vacancydb

In [3]:
answer = input('Очистить базу перед использованием?  [y/n]')
if answer == 'y':
    vacancydb.remove()

Очистить базу перед использованием?  [y/n]y


/home/ssv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
headers = {'Accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linu…) Gecko/20100101 Firefox/67.0'}

In [5]:
pages = int(input('Сколько страниц надо запросить   '))

Сколько страниц надо запросить   3


In [6]:
language = input('Какой язык?      ')

Какой язык?      Java


### HH.RU

In [7]:
df_list = []
for i in tqdm(range(pages)):
    vacancy = pd.DataFrame(columns=['Название вакансии', 'Зарплата', 'Ссылка'])
    headers = {'Accept': '*/*',
               'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0'}
    html = requests.get('https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&\
    search_field=name&text=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82'
                        + language + '&page=' + str(i), headers=headers).text
    soup = bs(html, 'html.parser')
    divs = soup.find_all('div', attrs={'data-qa': ['vacancy-serp__vacancy',
                                                   'vacancy-serp__vacancy vacancy-serp__vacancy_premium']})
    for div in divs:
        title = div.find(
            'a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text
        href = div.find(
            'a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
        salary = div.find(
            'div', attrs={'class': 'vacancy-serp-item__sidebar'}).text
        tmp_salary = re.findall(r'\b\d+\b',salary)
        min_salary = ''
        max_salary = ''
        if len(tmp_salary)>0:
            min_salary = int(tmp_salary[0] + tmp_salary[1])
        if len(tmp_salary)>3:
            max_salary = int(tmp_salary[2] + tmp_salary[3])


        vacancydb.insert({'Название вакансии': title,'min_salary':min_salary,
                          'max_salary':max_salary, 'Сайт':'hh.ru', 'Ссылка': href})


100%|██████████| 3/3 [00:10<00:00,  3.73s/it]


### SuperJob

In [8]:
if language =='':
    language = input('Для сайта superjob необходимо указать язык программирования для вакансии      ')

In [9]:
for i in tqdm(range(pages)):
    vacancy = pd.DataFrame(columns= ['Название вакансии', 'Зарплата', 'Ссылка'])
    headers = {'Accept': '*/*',
           'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:67.0) Gecko/20100101 Firefox/67.0'}
    html = requests.get('https://www.superjob.ru/vakansii/programmist-' + language + '.html?geo[c][0]=1&page=' + str(i+1),\
                        headers=headers).text
    soup = bs(html, 'lxml')
    divs = soup.find_all('div', attrs={'class':'_3zucV _2GPIV i6-sc _3VcZr'})
    for div in divs:
        title = div.find('div', attrs={'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
        start = str(div).find('<div class="_2g1F-"><a')
        tmp = str(div)[start:]
        str_1 = tmp[:tmp.find('target="_blank">')]
        href = 'https://www.superjob.ru' + re.findall('href="(.+)"', str_1)[0]
        salary = div.find('span', attrs={'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})\
        .text.replace(u'\xa0', ' ')
        tmp_salary = re.findall(r'\b\d+\b',salary)
        min_salary = ''
        max_salary = ''
        if len(tmp_salary)>0:
            min_salary = int(tmp_salary[0] + tmp_salary[1])
        if len(tmp_salary)>3:
            max_salary = int(tmp_salary[2] + tmp_salary[3])
        vacancydb.insert({'Название вакансии': title,'min_salary':min_salary,
                          'max_salary':max_salary,'Сайт': 'superjob.ru',  'Ссылка': href})


100%|██████████| 3/3 [00:17<00:00,  5.82s/it]


In [10]:
payment = int(input('Показать вакансии с зарплатой от '))

Показать вакансии с зарплатой от 100000


In [11]:
objects = vacancydb.find({'min_salary':{'$gt':payment}})
for i in objects:
    pprint(i)

{'_id': ObjectId('5d5bc0e324fc55da3db0fdaa'),
 'max_salary': '',
 'min_salary': 120000,
 'Название вакансии': 'Java-разработчик',
 'Сайт': 'hh.ru',
 'Ссылка': 'https://novosibirsk.hh.ru/vacancy/33108761?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20java'}
{'_id': ObjectId('5d5bc0e324fc55da3db0fdab'),
 'max_salary': '',
 'min_salary': 200000,
 'Название вакансии': 'Senior java developer',
 'Сайт': 'hh.ru',
 'Ссылка': 'https://novosibirsk.hh.ru/vacancy/32893126?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20java'}
{'_id': ObjectId('5d5bc0e324fc55da3db0fdac'),
 'max_salary': '',
 'min_salary': 150000,
 'Название вакансии': 'Специалист по тестированию',
 'Сайт': 'hh.ru',
 'Ссылка': 'https://novosibirsk.hh.ru/vacancy/32349434?query=%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%20java'}
{'_id': ObjectId('5d5bc0e324fc55da3db0fdad'),
 'max_salary': '',
 'min_salary': 150000,
 'Название вакансии': 'Big Data Engineer',